In [1]:
import gc
import numpy as np
import pandas as pd

# Reduce copies in pandas operations
pd.set_option('mode.copy_on_write', True)

In [2]:
# Cell 2
candidates = pd.read_parquet('../data/outputs/candidates.parquet')

# Downcast and categorize to save memory
candidates['customer_id'] = candidates['customer_id'].astype('int64')
candidates['article_id']  = candidates['article_id'].astype('int32')
candidates['value'] = candidates['value'].astype('float32')
if 'window_type' in candidates.columns:
    candidates['window_type'] = candidates['window_type'].astype('category')

# ~ 34gb of ram usage, 1 min 14s

In [3]:
# Cell 3
tx = pd.read_csv(
    '../data/input_data/transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id'],
    dtype={'t_dat': 'string', 'customer_id': 'string', 'article_id': 'int32'},
)
tx['customer_id'] = tx['customer_id'].str[-16:].apply(lambda h: np.int64(np.uint64(int(h,16))))
tx['t_dat'] = pd.to_datetime(tx['t_dat'])
last_ts = tx['t_dat'].max()

# cutoff for history – same as in candidate pipelines
cut_ts = last_ts - pd.Timedelta(days=7)

# Keep only minimal columns needed going forward
tx_full = tx[tx['t_dat'] <= cut_ts][['customer_id', 'article_id', 't_dat']].copy()

# 8-week history before cut_ts (for recent features)
history_start = cut_ts - pd.Timedelta(days=56)
tx_hist = tx[(tx['t_dat'] > history_start) & (tx['t_dat'] <= cut_ts)][['customer_id','article_id','t_dat']].copy()

# tx no longer needed
del tx
gc.collect()

week_start = cut_ts - pd.Timedelta(days=7)
month_start = cut_ts - pd.Timedelta(days=28)

# Recent counts (active users)
cust_1w = (
    tx_hist[tx_hist['t_dat'] > week_start]
    .groupby('customer_id')['article_id']
    .size()
    .reset_index(name='cust_purchases_1w')
)
cust_4w = (
    tx_hist[tx_hist['t_dat'] > month_start]
    .groupby('customer_id')['article_id']
    .size()
    .reset_index(name='cust_purchases_4w')
)

# tx_hist no longer needed
del tx_hist
gc.collect()

# Load customer meta (only needed columns)
cust_meta = pd.read_csv(
    '../data/input_data/customers.csv',
    usecols=['customer_id','age','postal_code','club_member_status','fashion_news_frequency'],
    dtype={'customer_id': 'string'},
)

# Convert customer_id to int to match tx / candidates
cust_meta['customer_id'] = cust_meta['customer_id'].str[-16:].apply(lambda h: np.int64(np.uint64(int(h, 16))))

# Basic cleaning / encoding (categorize strings to reduce memory)
cust_meta['age'] = cust_meta['age'].fillna(-1).astype('float32')
for col in ['postal_code', 'club_member_status', 'fashion_news_frequency']:
    if col in cust_meta.columns:
        cust_meta[col] = cust_meta[col].astype('category')

# Last purchase date for each customer (any article) – from history only
cust_last_purchase = (
    tx_full.sort_values('t_dat')
          .drop_duplicates(['customer_id'], keep='last')
          [['customer_id', 't_dat']]
          .rename(columns={'t_dat': 'customer_last_purchase_date'})
)

# Load prices to compute price-based aggregates (history only)
tx_price = pd.read_csv(
    '../data/input_data/transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id', 'price'],
    dtype={'t_dat': 'string', 'customer_id': 'string', 'article_id': 'int32', 'price': 'float32'},
)
tx_price['customer_id'] = tx_price['customer_id'].str[-16:].apply(lambda h: np.int64(np.uint64(int(h, 16))))
tx_price['t_dat'] = pd.to_datetime(tx_price['t_dat'])

tx_price_hist = tx_price[tx_price['t_dat'] <= cut_ts].copy()

# Article-level price stats (history)
art_price_agg = (
    tx_price_hist.groupby('article_id')
                 .agg(article_mean_price=('price', 'mean'))
                 .reset_index()
)

# Customer-level price stats (history)
cust_price_agg = (
    tx_price_hist.groupby('customer_id')
                 .agg(customer_mean_price=('price', 'mean'))
                 .reset_index()
)

del tx_price, tx_price_hist
gc.collect()

# ~ 18gb of ram usage, 4 min 46s

22

In [4]:
# Cell 4 – customer-level cumulative features (all history up to cut_ts)
cust_agg = (
    tx_full.groupby('customer_id')
      .agg(
          customer_total_purchases=('article_id', 'size'),
          customer_unique_articles=('article_id', 'nunique'),
      )
      .reset_index()
)

In [5]:
# Cell 5 – article-level cumulative features (all history up to cut_ts)
art_agg = (
    tx_full.groupby('article_id')
      .agg(
          article_total_purchases=('customer_id', 'size'),
          article_unique_customers=('customer_id', 'nunique'),
      )
      .reset_index()
)

# Load only needed article meta columns
art_meta = pd.read_csv(
    '../data/input_data/articles.csv',
    usecols=[
        'article_id',
        'product_type_no',
        'product_group_name',
        'index_code',
        'section_no',
        'graphical_appearance_no',
        'colour_group_code',
        'perceived_colour_value_id',
        'perceived_colour_master_id',
        'index_group_no',
        'garment_group_no',
    ],
    dtype={'article_id': 'int32'}
)


# Encode categorical fields
for col in ['product_group_name', 'index_code']:
    art_meta[col] = art_meta[col].astype('category').cat.codes.astype('int16')

# Downcast numeric codes to smaller ints where safe
for col in [
    'product_type_no','section_no','graphical_appearance_no','colour_group_code',
    'perceived_colour_value_id','perceived_colour_master_id','index_group_no','garment_group_no'
]:
    if col in art_meta.columns:
        art_meta[col] = pd.to_numeric(art_meta[col], downcast='integer')

In [6]:
# Cell 6 – last purchase per (customer, article) using all history
last_purchase = (
    tx_full.sort_values('t_dat')
      .drop_duplicates(['customer_id', 'article_id'], keep='last')
      [['customer_id', 'article_id', 't_dat']]
      .rename(columns={'t_dat': 'last_purchase_date'})
)

# tx_full no longer needed after last_purchase is built
del tx_full
gc.collect()

0

In [7]:
# Cell 7 – merge features onto candidates
features = candidates.merge(
    cust_agg, on='customer_id', how='left'
)
features = features.merge(
    cust_1w, on='customer_id', how='left'
)
features = features.merge(
    cust_4w, on='customer_id', how='left'
)
features = features.merge(
    art_agg, on='article_id', how='left'
)
features = features.merge(
    last_purchase, on=['customer_id', 'article_id'], how='left'
)
features = features.merge(
    cust_meta, on='customer_id', how='left'
)
features = features.merge(
    art_meta, on='article_id', how='left'
)
features = features.merge(
    cust_last_purchase, on='customer_id', how='left'
)
features = features.merge(
    art_price_agg, on='article_id', how='left'
)
features = features.merge(
    cust_price_agg, on='customer_id', how='left'
)

# Free merged sources early
del cust_agg, cust_1w, cust_4w, art_agg, last_purchase
del cust_meta, art_meta, cust_last_purchase, art_price_agg, cust_price_agg
gc.collect()

# ~ 32 gb of ram usage, 4 min 46s

0

In [8]:
# Debug coverage: candidates vs features
print("candidates – rows:", len(candidates))
print("candidates – unique customers:", candidates['customer_id'].nunique())

print("features – rows:", len(features))
print("features – unique customers:", features['customer_id'].nunique())

candidates – rows: 262416062


candidates – unique customers: 1371980
features – rows: 262416062
features – unique customers: 1371980


In [9]:
# Cell 8 – recency and encoding
del candidates
gc.collect()

features['days_since_last_purchase'] = (
    (cut_ts - features['last_purchase_date'])
    .dt.days
    .astype('float32')
)

# Customer recency (any purchase)
features['customer_days_since_last_purchase'] = (
    (cut_ts - features['customer_last_purchase_date'])
    .dt.days
    .astype('float32')
)

# Drop datetime cols after deriving recencies
features = features.drop(columns=['last_purchase_date','customer_last_purchase_date'])

for col in [
    'customer_total_purchases',
    'customer_unique_articles',
    'article_total_purchases',
    'article_unique_customers',
    'cust_purchases_1w',
    'cust_purchases_4w',
    'days_since_last_purchase',
    'customer_days_since_last_purchase',
    'article_mean_price',
    'customer_mean_price',
]:
    features[col] = features[col].fillna(0).astype('float32')

# Make missing age explicit as -1 (same as in cust_meta)
features['age'] = features['age'].fillna(-1).astype('float32')

for col in ['club_member_status', 'fashion_news_frequency']:
    if col in features.columns:
        features[col] = features[col].astype('category').cat.codes.astype('int16')

# Postal code can have high cardinality; encode as category codes
if 'postal_code' in features.columns:
    features['postal_code'] = features['postal_code'].astype('category').cat.codes.astype('int32')

# Encode window_type now and drop the string column to save memory
if 'window_type' in features.columns:
    features['window_type_code'] = features['window_type'].astype('category').cat.codes.astype('int16')
    features = features.drop(columns=['window_type'])

gc.collect()

# ~ 25gb of ram usage, 18s

36

In [10]:
# Cell 9
import json

feature_cols_order = [c for c in features.columns if c not in ['customer_id','article_id']]
meta_payload = {
    "feature_cols": ['customer_id','article_id'] + feature_cols_order,
    "timestamp": pd.Timestamp.utcnow().isoformat(),
}
with open("../data/outputs/feature_cols.json","w") as f:
    json.dump(meta_payload, f)

In [11]:
# Cell 10 – encode window_type and save
features.to_parquet('../data/outputs/features.parquet', index=False)

# Free memory
del features
gc.collect()

# ~ 31gb of ram usage, 2 min 44s

0